In [1]:
import time
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

from torchvision import transforms, models
from torch.utils.data import DataLoader
from src.products_dataset import ProductsDataset
from src.multilabel_model import MultilabelModel

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [14]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
#         transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
#         transforms.RandomRotation(8),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])])
}

datasets = {mode: ProductsDataset(xlsx_filepath='./data/products-{}.xlsx'.format(mode),
                                  root_dir='./data/images',
                                  transform=data_transforms[mode])
            for mode in ['train', 'val', 'test']}

dataloaders = {mode: DataLoader(datasets[mode], batch_size=64, shuffle=True, num_workers=8)
               for mode in ['train', 'val']}

dataloaders['test'] = DataLoader(datasets['test'], batch_size=64, shuffle=True, num_workers=8)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: {}'.format(device))

cuda:0


In [3]:
model = MultilabelModel()

In [4]:
model = model.to(device)

class_weights = datasets['train'].classes_weights['category']
cat_weights = torch.Tensor(list(class_weights.values())).to(device)
category_criterion = nn.CrossEntropyLoss(weight=cat_weights)

class_weights = datasets['train'].classes_weights['condition']
cond_weights = torch.Tensor(list(class_weights.values())).to(device)
condition_criterion = nn.CrossEntropyLoss(weight=cond_weights)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [5]:
def train_model(model, dataloaders, category_criterion, condition_criterion, optimizer, num_epochs=10):
    tic = time.time()
    
    train_history = {}

    best_model_weights = copy.deepcopy(model.state_dict())
    best_cond_acc = 0.0
    best_cat_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for mode in ['train', 'val']:
            train_history[mode] = {}
            train_history[mode]['loss'] = []
            train_history[mode]['accuracy'] = []

            if mode == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            cat_accuracy = 0
            cond_accuracy = 0
            
            for batch in dataloaders[mode]:
                inputs = batch['image'].to(device)
                gt_categories = batch['category'].to(device)
                gt_conditions = batch['condition'].to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(mode == 'train'):
                    out_categories, out_conditions = model(inputs)

                    _, cat_predictions = torch.max(out_categories, 1)
                    _, cond_predictions = torch.max(out_conditions, 1)

                    loss_category = category_criterion(out_categories, gt_categories)
                    loss_condition = condition_criterion(out_conditions, gt_conditions)
                    loss = 0.8 * loss_category + 1.2 * loss_condition
                    
                    if mode == 'train':
                        loss.backward()
                        optimizer.step()
            
                running_loss += loss.item() * inputs.size(0)
                cat_accuracy += torch.sum(cat_predictions == gt_categories.data)
                cond_accuracy += torch.sum(cond_predictions == gt_conditions.data)
            
            epoch_loss = running_loss / len(dataloaders[mode].dataset)
            epoch_cat_acc = cat_accuracy.double() / len(dataloaders[mode].dataset)
            epoch_cond_acc = cond_accuracy.double() / len(dataloaders[mode].dataset)
            
            train_history[mode]['loss'].append(epoch_loss)
            train_history[mode]['accuracy'].append((epoch_cat_acc, epoch_cond_acc))
            
            print('{} loss: {:.4f}, categories acc: {:.4f}, conditions acc: {:.4f}'.format(
                mode, epoch_loss, epoch_cat_acc, epoch_cond_acc))

            if mode == 'val' and epoch_cond_acc > best_cond_acc and epoch_cat_acc > best_cat_acc:
                best_cond_acc = epoch_cond_acc
                best_cat_acc = epoch_cat_acc
                best_model_weights = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - tic
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best accuracies: condition: {:4f}, category: {:4f}'.format(best_cond_acc, best_cat_acc))

    model.load_state_dict(best_model_weights)

    return model, train_history

In [6]:
model, train_history = train_model(model, dataloaders, category_criterion,
                                   condition_criterion, optimizer, num_epochs=10)

Epoch 0/9
----------
train loss: 3.7547, categories acc: 0.3890, conditions acc: 0.2910
val loss: 3.2679, categories acc: 0.4527, conditions acc: 0.2827

Epoch 1/9
----------
train loss: 2.8595, categories acc: 0.5518, conditions acc: 0.3895
val loss: 3.1764, categories acc: 0.5700, conditions acc: 0.3964

Epoch 2/9
----------
train loss: 2.1655, categories acc: 0.6069, conditions acc: 0.4747
val loss: 3.1520, categories acc: 0.5564, conditions acc: 0.3682

Epoch 3/9
----------
train loss: 1.7300, categories acc: 0.6481, conditions acc: 0.5387
val loss: 3.1919, categories acc: 0.6009, conditions acc: 0.4200

Epoch 4/9
----------
train loss: 1.4466, categories acc: 0.6905, conditions acc: 0.6102
val loss: 3.6319, categories acc: 0.6227, conditions acc: 0.4627

Epoch 5/9
----------
train loss: 1.2049, categories acc: 0.7396, conditions acc: 0.6607
val loss: 3.7203, categories acc: 0.6327, conditions acc: 0.4900

Epoch 6/9
----------
train loss: 1.0087, categories acc: 0.7757, conditions 

In [12]:
def test_model(model, dataloader):
    tic = time.time()

    model.eval()

    running_cat_acc = 0
    running_cond_acc = 0

    for batch in dataloader:
        inputs = batch['image'].to(device)
        gt_categories = batch['category'].to(device)
        gt_conditions = batch['condition'].to(device)

        with torch.no_grad():
            out_categories, out_conditions = model(inputs)

            _, cat_predictions = torch.max(out_categories, 1)
            _, cond_predictions = torch.max(out_conditions, 1)

        running_cat_acc += torch.sum(cat_predictions == gt_categories.data)
        running_cond_acc += torch.sum(cond_predictions == gt_conditions.data)

    cat_accuracy = running_cat_acc.double() / len(dataloader.dataset)
    cond_accuracy = running_cond_acc.double() / len(dataloader.dataset)

    print('Categories accuracy: {:.4f}'.format(cat_accuracy))
    print('Conditions accuracy: {:.4f}'.format(cond_accuracy))

    time_elapsed = time.time() - tic
    print('Testing complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return model

In [15]:
tested_model = test_model(model, dataloaders['test'])

categories accuracy: 0.6312, conditions accuracy: 0.4846
Testing complete in 0m 5s


#### categories accuracy: 0.6312, conditions accuracy: 0.4846

In [17]:
torch.save(model.state_dict(), './weights/resnet50-10epochs-63-48.pt')